In [ ]:
!pip install diffusers
!pip install open_clip_torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 17.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 26.1 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, transforms
import torch.nn as nn
import open_clip
from diffusers import AudioLDMPipeline
from PIL import Image
import scipy
import matplotlib.pyplot as plt
import numpy as np
import json

In [ ]:
# CLIP, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
# tokenizer = open_clip.get_tokenizer('ViT-B-32')

CLIP, _, preprocess = open_clip.create_model_and_transforms('ViT-B-32', pretrained='laion2b_s34b_b79k')
tokenizer = open_clip.get_tokenizer('ViT-B-32')
CLIP.to("cuda")
repo_id = "cvssp/audioldm-l-full"
CLAP = AudioLDMPipeline.from_pretrained(repo_id, torch_dtype=torch.float16).to("cuda")


open_clip_pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

model_index.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

vocoder/model.safetensors not found


Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

tokenizer/tokenizer_config.json:   0%|          | 0.00/424 [00:00<?, ?B/s]

...

diffusion_pytorch_model.bin:   0%|          | 0.00/2.96G [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/221M [00:00<?, ?B/s]

diffusion_pytorch_model.bin:   0%|          | 0.00/222M [00:00<?, ?B/s]

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
def get_embeds(data):
  clip_tokens = open_clip.tokenize(data).to("cuda")

  with torch.no_grad(), torch.cuda.amp.autocast():
    clip_embeds = CLIP.encode_text(clip_tokens).to("cuda")

    clap_embeds = CLAP._encode_prompt(
        prompt = data,
        num_waveforms_per_prompt = 1,
        do_classifier_free_guidance = False,
        device="cuda"
    ).to("cuda")

  return clip_embeds.float(), clap_embeds.float()

def translate_image(path, model, CLIP, AudioLDM):
    model.eval()
    image = preprocess(Image.open(path)).unsqueeze(0)

    with torch.no_grad(), torch.cuda.amp.autocast():
        image_embed = CLIP.encode_image(image.to("cuda"))
        # torch.nn.functional.normalize(image_embed, p=2, dim=1)

    image_embed = image_embed.float().to("cuda")
    image_embed = model(image_embed)

    print(image_embed.shape)
    print(f"Max = {image_embed.max()}")
    print(f"Min = {image_embed.min()}")
    print(f"Mean = {torch.mean(image_embed)}")

    negative_prompt_embeds = AudioLDM._encode_prompt(
        prompt = "Low quality",
        device = "cuda",
        num_waveforms_per_prompt = 1,
        do_classifier_free_guidance = False
    )

    audio = AudioLDM(
        negative_prompt_embeds=negative_prompt_embeds,
        prompt_embeds=image_embed,
        num_inference_steps=200,
        audio_length_in_s=10.0,
        guidance_scale=7.5
      ).audios[0]

    # scipy.io.wavfile.write(f"{path[:-4]}.wav", rate=16000, data=audio)
    scipy.io.wavfile.write(f"girl_linear3_euclid_contrastive_512_e0.wav", rate=16000, data=audio)

In [ ]:
class LinearAligner(nn.Module):
  def __init__(self, clip_embed_dim, clap_embed_dim):
    super().__init__()
    self.linear = nn.Linear(clip_embed_dim, clap_embed_dim)

  def forward(self, x):
    # Pass through linear layer 1
    return torch.clamp(self.linear(x), min=-0.3, max=0.3)

In [ ]:
class LinearAligner2(nn.Module):
  def __init__(self, clip_embed_dim, clap_embed_dim):
    super().__init__()
    self.linear = nn.Linear(clip_embed_dim, clap_embed_dim)
    self.linear2 = nn.Linear(clap_embed_dim, clap_embed_dim)

  def forward(self, x):
    x = self.linear(x)
    x = nn.Tanh()(x)
    return torch.clamp(self.linear2(x), min=-0.3, max=0.3)

In [ ]:
class LinearAligner3(nn.Module):
  def __init__(self, clip_embed_dim, clap_embed_dim):
    super().__init__()
    self.ff1 = nn.Linear(clip_embed_dim, clap_embed_dim)
    self.ff2 = nn.Linear(clap_embed_dim, clap_embed_dim)
    self.ff3 = nn.Linear(clap_embed_dim, clap_embed_dim)
    self.ff4 = nn.Linear(clap_embed_dim, clap_embed_dim)

  def forward(self, x):
    x = self.ff1(x)
    x = nn.Tanh()(x)
    x = self.ff2(x)
    x = nn.Tanh()(x)
    x = self.ff3(x)
    x = nn.Tanh()(x)
    return torch.clamp(self.ff4(x), min=-0.2, max=0.2)

In [ ]:
model = LinearAligner3(512, 512)
model.to("cuda")
model = nn.DataParallel(model)
model.eval()

checkpoint = torch.load("/content/linear3_contrastive_euclid_512_epoch_0.pt")
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [ ]:
translate_image("/content/girl.png", model, CLIP, CLAP)

torch.Size([1, 512])
Max = 0.12036679685115814
Min = -0.11413049697875977
Mean = 1.0430157999508083e-05


  0%|          | 0/200 [00:00<?, ?it/s]

In [ ]:
clips, claps = get_embeds("A beach")
print(clips.shape)
print(claps.shape)

print(torch.min(clips))
print(torch.max(clips))

print(torch.min(claps))
print(torch.max(claps))

torch.Size([1, 512])
torch.Size([1, 512])
tensor(-4.0508, device='cuda:0')
tensor(0.9116, device='cuda:0')
tensor(-0.1248, device='cuda:0')
tensor(0.1375, device='cuda:0')
